# Finetuning RoBERTa for NER: Train Model
 

***

## Imports

In [243]:
from transformers import (BertTokenizerFast,
                          RobertaTokenizerFast,
                          AutoTokenizer,
                          BertForTokenClassification,
                          RobertaForTokenClassification,
                          DataCollatorForTokenClassification, 
                          AutoModelForTokenClassification, 
                          TrainingArguments, Trainer)
from datasets import load_dataset, load_metric, concatenate_datasets, DatasetDict
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import dill as pickle
import torch
import math
import os

## Load Dataset

In [244]:
data_path = "./data/dataset_processed.pkl"
with open(data_path, 'rb') as pickle_file:
    dataset = pickle.load(file=pickle_file)

In [245]:
dataset["train"][0]

{'tokens': ["''", 'Super', 'World', 'of', 'Sports', "''", "'"],
 'ner_tags': [0, 3, 4, 4, 4, 0, 0],
 'langs': ['en', 'en', 'en', 'en', 'en', 'en', 'en'],
 'spans': ['ORG: Super World of Sports'],
 'input_ids': [5106, 4265, 6661, 111, 39170, 5106, 242],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1],
 'labels': [0, 3, 4, 4, 4, 0, 0]}

## Load Model and Tokenizer

Information about model variants can be found here: https://huggingface.co/docs/transformers/model_doc/roberta

Load Model which can be finetuned:

In [246]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [247]:
label_list = dataset["train"].features[f"ner_tags"].feature.names

In [248]:
model_name = "xlm-roberta-large" #"bert-base-multilingual-cased" #xlm-roberta-large
tokenizer = AutoTokenizer.from_pretrained(f"{model_name}", add_prefix_space=True) #AutoTokenizer(use_fast = True)
model = AutoModelForTokenClassification.from_pretrained(f"{model_name}", num_labels=len(label_list))

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /home/pop529700/.cache/huggingface/hub/models--xlm-roberta-large/snapshots/b2a6150f8be56457baf80c74342cc424080260f0/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-large",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading file s

## Define Data Collator

In [249]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Define Trainer

About the Model:

see https://github.com/huggingface/transformers/blob/v4.21.1/src/transformers/modeling_utils.py#L829

In [250]:
print("Parameters:", model.num_parameters())
print("Expected Input Dict:", model.main_input_name )

# Estimate FLOPS needed for one training example
sample = dataset["train"][0]
sample["input_ids"] = torch.Tensor(sample["input_ids"])
flops_est = model.floating_point_ops(input_dict = sample, exclude_embeddings = False)

print("FLOPS needed per Training Sample:", flops_est )

Parameters: 558848007
Expected Input Dict: input_ids
FLOPS needed per Training Sample: 23471616294


In [251]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

**Define Optimizer:**

See https://huggingface.co/docs/transformers/main_classes/optimizer_schedules#transformers.Adafactor

In [252]:
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

num_epochs = 5
batch_size = 16
num_reports = 5

# A training step is one gradient update. In one step batch_size examples are processed.
# An epoch consists of one full cycle through the training data. 
# This is usually many steps. As an example, if you have 2,000 images and use
# a batch size of 10 an epoch consists of:
gpu_count = torch.cuda.device_count()
num_steps = (len(dataset["train"]) / batch_size / gpu_count) * num_epochs

optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-6, weight_decay=0.01, no_deprecation_warning= True)

scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0, 
    num_training_steps= num_steps 
)

print("Steps:", num_steps)

Steps: 156.25


**Define Log and Eval Interval:**

In [253]:
report_steps = math.floor(num_steps / num_reports)
print("Eval interval:", report_steps)

Eval interval: 31


**Define Metrics:**

See https://huggingface.co/course/chapter7/2#metrics

In [254]:
metric = load_metric("seqeval")

In [255]:
example = dataset["train"][150]
labels = [label_list[i] for i in example[f"labels"]]
metric.compute(predictions=[labels], references=[labels])

{'LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

Set correct class labels:

In [256]:
label_names = dataset["train"].features[f"ner_tags"].feature.names

id2label = {id : label for id, label in enumerate(label_names)}
label2id = {label: id for id, label in enumerate(label_names)}

model.config.id2label = id2label
model.config.label2id = label2id

Define callback function to evaluate the model:

In [257]:
label_names = model.config.id2label

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    true_labels = [[label_names[l] for l in label  if l != -100] for label in labels]
    #true_predictions = [model.config.id2label[t.item()] for t in predictions]
    
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label)  if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

**Remove unnecessary columns:**

In [258]:
dataset = dataset.remove_columns(["tokens", "ner_tags", "langs", "spans"])

**Set further Training Arguments:**

See https://huggingface.co/docs/transformers/v4.21.2/en/main_classes/trainer#transformers.TrainingArguments

In [259]:
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy = "steps",
    save_steps = report_steps,
    remove_unused_columns = True,
    evaluation_strategy="steps",
    eval_steps = report_steps,
    #load_best_model_at_end=True,
    logging_strategy = "steps",
    logging_steps = report_steps,
    #learning_rate= 2e-5,
    #auto_find_batch_size = True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    #gradient_accumulation_steps=4,
    #optim="adamw_torch",
    num_train_epochs=num_epochs,
    #weight_decay=0.01,
    report_to="none",
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    optimizers=(optimizer, scheduler),
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
Using cuda_amp half precision backend


## Train Model

GPU used by Kaggle: https://www.nvidia.com/de-de/data-center/tesla-p100/

In [260]:
!nvidia-smi

Sat Jan  7 13:07:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          Off  | 00000000:01:00.0 Off |                    0 |
|  0%   48C    P0    81W / 300W |  34810MiB / 46068MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A40          Off  | 00000000:43:00.0 Off |                    0 |
|  0%   

In [261]:
trainer.train()

***** Running training *****
  Num examples = 1000
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 160
  Number of trainable parameters = 558848007
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/pop529700/.pyenv/versions/3.10.8/envs/transformers/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
31,1.788100,1.475708,0.067762,0.027273,0.038892,0.478928
62,1.382300,1.148932,0.144931,0.235124,0.179326,0.605034
93,1.077200,0.880566,0.277396,0.316942,0.295853,0.696704
124,0.881500,0.787374,0.341313,0.365289,0.352894,0.737783
155,0.809300,0.722780,0.379388,0.419835,0.398588,0.759347


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-31
Configuration saved in ./results/checkpoint-31/config.json
Model weights saved in ./results/checkpoint-31/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-31/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-31/special_tokens_map.json
/home/pop529700/.pyenv/versions/3.10.8/envs/transformers/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-62
Configuration saved in ./results/checkpoint-62/config.json
Model weights saved in ./results/checkpoint-62/pytorch_model.bin
tokenizer config file saved in ./r

TrainOutput(global_step=160, training_loss=1.174519120156765, metrics={'train_runtime': 118.0347, 'train_samples_per_second': 42.36, 'train_steps_per_second': 1.356, 'total_flos': 387727231040688.0, 'train_loss': 1.174519120156765, 'epoch': 5.0})

In [262]:
eval_results = trainer.evaluate()
print(f"Eval Loss: {eval_results['eval_loss']}")

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 32


Eval Loss: 0.7214924097061157


**Saving the fine tuned model & tokenizer:**

In [263]:
trainer.save_model(f'./results/checkpoint-final/')

Saving model checkpoint to ./results/checkpoint-final/
Configuration saved in ./results/checkpoint-final/config.json
Model weights saved in ./results/checkpoint-final/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-final/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-final/special_tokens_map.json


**Save Training History:**

In [264]:
data_path = "./results/checkpoint-final/training_args.pkl"
with open(data_path, 'wb') as pickle_file:
    pickle.dump(obj = trainer.args, file=pickle_file)

In [265]:
data_path = "./results/checkpoint-final/training_history.pkl"
with open(data_path, 'wb') as pickle_file:
    pickle.dump(obj = trainer.state, file=pickle_file)

**Calculate Accuracy:**

In [266]:
predictions, labels, _ = trainer.predict(dataset["test"])
predictions = np.argmax(predictions, axis=-1)

***** Running Prediction *****
  Num examples = 1000
  Batch size = 32


In [267]:
true_labels = [
    [label_names[l] for l in label  if l != -100] 
    for label in labels
]

true_predictions = [
    [label_names[p] for (p, l) in zip(prediction, label)  if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
pprint(results)

{'LOC': {'f1': 0.42008691453404157,
         'number': 880,
         'precision': 0.36523929471032746,
         'recall': 0.4943181818181818},
 'ORG': {'f1': 0.24005681818181818,
         'number': 758,
         'precision': 0.26,
         'recall': 0.22295514511873352},
 'PER': {'f1': 0.5234228607120549,
         'number': 782,
         'precision': 0.5115995115995116,
         'recall': 0.5358056265984654},
 'overall_accuracy': 0.7610692030173828,
 'overall_f1': 0.402755905511811,
 'overall_precision': 0.38458646616541353,
 'overall_recall': 0.42272727272727273}
